In [1]:
%load_ext autoreload
%autoreload complete

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from matplotlib_inline.backend_inline import set_matplotlib_formats
from protest_impact.util import cache, project_root
from tqdm.notebook import tqdm

set_matplotlib_formats("svg")

In [2]:
path = project_root / "protest_impact/data/protests/german_protest_registrations"
df = pd.read_csv(path / "all-protests.csv", parse_dates=["Datum"])
df.head()

,Datum,Uhrzeit,Ort,Teilnehmer,Veranstalter,Kategorie,Thema,Stadt,Ende,Beginn,Datum der Anmeldung,Teilnehmer (tatsächlich),Stichwörter,Bundesland
0,2022-06-02,14:00 - 17:00,Parkplatz Hermgesberg,70.0,Initiative Osterholz bleibt,Aufzug,Waldspaziergang - Osterholz bleibt!,Wuppertal,NaN,NaN,NaN,NaN,NaN,Nordrhein-Westfalen
1,2022-03-06,14:00 - 17:00,Parkplatz Hermgesberg,70.0,Initiative Osterholz bleibt,Aufzug,Waldspaziergang - Osterholz bleibt!,Wuppertal,NaN,NaN,NaN,NaN,NaN,Nordrhein-Westfalen
2,2022-04-03,14:00 - 17:00,Parkplatz Hermgesberg,70.0,Initiative Osterholz bleibt,Aufzug,Waldspaziergang - Osterholz bleibt!,Wuppertal,NaN,NaN,NaN,NaN,NaN,Nordrhein-Westfalen
3,2022-06-05,14:00 - 17:00,Parkplatz Hermgesberg,70.0,Initiative Osterholz bleibt,Aufzug,Waldspaziergang - Osterholz bleibt!,Wuppertal,NaN,NaN,NaN,NaN,NaN,Nordrhein-Westfalen
4,2022-01-08,14:00 - 16:00,Bahnhof,15.0,Arbeiterkommunistische Partei des Iran,Versammlung,Jahrestag der Tötung von 167 Passagieren des F...,Wuppertal,NaN,NaN,NaN,NaN,NaN,Nordrhein-Westfalen


In [3]:
df = df.rename(columns={"Datum": "date", "Stadt": "city", "Bundesland": "region"})

In [4]:
df.groupby(["city"])["Veranstalter"].nunique().sort_values(ascending=False)

city
München      925
Erfurt       445
Wiesbaden    372
Wuppertal     62
Berlin         0
Bremen         0
Dresden        0
Karlsruhe      0
Kiel           0
Köln           0
Magdeburg      0
Mainz          0
Potsdam        0
Name: Veranstalter, dtype: int64

In [5]:
df.groupby(["city"])["Thema"].nunique().sort_values(ascending=False)

city
Berlin       14832
München       3292
Bremen        1490
Magdeburg     1452
Köln          1412
Mainz          951
Erfurt         830
Karlsruhe      676
Potsdam        602
Dresden        556
Wiesbaden      495
Kiel           445
Wuppertal      143
Name: Thema, dtype: int64

In [6]:
df["year"] = df["date"].dt.year.astype(str).apply(lambda x: x[:-2])

In [7]:
agg_df = df.groupby(["region", "city", "year"]).size().unstack().fillna(0).astype(int)
agg_df = agg_df.drop(columns=["2023", "n"])
agg_df

year                           2015  2016  2017  2018  2019  2020  2021  2022
region              city                                                     
Baden-Württemberg   Karlsruhe     0     0     0     0     0     0   490   467
Bayern              München       0     0     0  1145  1170  1364  1983  1169
Berlin              Berlin        0     0     0  4290  5405  5890  6207  6442
Brandenburg         Potsdam       0     0     0     0   214   238   287   326
Bremen              Bremen        0     0     0     0   654   631   623   721
Hessen              Wiesbaden     0     0     0     0   159   235   304   256
Nordrhein-Westfalen Köln          0     0     0  1037     0     0     0  1551
                    Wuppertal     0     0     0     0     0     0     0   209
Rheinland-Pfalz     Mainz         0     0     0   255   248   305   312   411
Sachsen             Dresden       0     0     0     0     0   164   292   405
Sachsen-Anhalt      Magdeburg   215   167   135   175   222   312   411   435
Schleswig-Holstein  Kiel          0     0     0     0     0     0   272   341
Thüringen           Erfurt        0     0     0     0   305   285   331   351

In [8]:
agg_df[["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]].sum()

year
2015      215
2016      167
2017      135
2018     6902
2019     8377
2020     9424
2021    11512
2022    13084
dtype: int64

In [9]:
agg_df[["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]].sum(axis=1)

region               city     
Baden-Württemberg    Karlsruhe      957
Bayern               München       6831
Berlin               Berlin       28234
Brandenburg          Potsdam       1065
Bremen               Bremen        2629
Hessen               Wiesbaden      954
Nordrhein-Westfalen  Köln          2588
                     Wuppertal      209
Rheinland-Pfalz      Mainz         1531
Sachsen              Dresden        861
Sachsen-Anhalt       Magdeburg     2072
Schleswig-Holstein   Kiel           613
Thüringen            Erfurt        1272
dtype: int64

In [10]:
agg_df[["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]].sum().sum()

49816

In [11]:
# add column to agg_df whether or not the "Teilnehmer" column is available
agg_df["registrations"] = df.groupby(["region", "city"]).apply(
    lambda x: x["Teilnehmer"].mean() > 10
)
agg_df["observations"] = df.groupby(["region", "city"]).apply(
    lambda x: x["Teilnehmer (tatsächlich)"].mean() > 10
)
agg_df

year                           2015  2016  2017  2018  2019  2020  2021  2022  \
region              city                                                        
Baden-Württemberg   Karlsruhe     0     0     0     0     0     0   490   467   
Bayern              München       0     0     0  1145  1170  1364  1983  1169   
Berlin              Berlin        0     0     0  4290  5405  5890  6207  6442   
Brandenburg         Potsdam       0     0     0     0   214   238   287   326   
Bremen              Bremen        0     0     0     0   654   631   623   721   
Hessen              Wiesbaden     0     0     0     0   159   235   304   256   
Nordrhein-Westfalen Köln          0     0     0  1037     0     0     0  1551   
                    Wuppertal     0     0     0     0     0     0     0   209   
Rheinland-Pfalz     Mainz         0     0     0   255   248   305   312   411   
Sachsen             Dresden       0     0     0     0     0   164   292   405   
Sachsen-Anhalt      Magdeburg   215   167   135   175   222   312   411   435   
Schleswig-Holstein  Kiel          0     0     0     0     0     0   272   341   
Thüringen           Erfurt        0     0     0     0   305   285   331   351   

year                           registrations  observations  
region              city                                    
Baden-Württemberg   Karlsruhe           True         False  
Bayern              München             True         False  
Berlin              Berlin              True          True  
Brandenburg         Potsdam             True         False  
Bremen              Bremen             False         False  
Hessen              Wiesbaden           True         False  
Nordrhein-Westfalen Köln                True         False  
                    Wuppertal           True         False  
Rheinland-Pfalz     Mainz               True         False  
Sachsen             Dresden             True         False  
Sachsen-Anhalt      Magdeburg           True          True  
Schleswig-Holstein  Kiel                True         False  
Thüringen           Erfurt             False         False

In [12]:
agg_df["incl?"] = agg_df["registrations"] & (agg_df["2020"] > 0)

In [13]:
from protest_impact.data import german_regions


def is_capital(city, region):
    return (
        len([a for a in german_regions if a["name"] == region and a["capital"] == city])
        == 1
    )


agg_df["capital"] = agg_df.apply(lambda x: is_capital(x.name[1], x.name[0]), axis=1)
agg_df

year                           2015  2016  2017  2018  2019  2020  2021  2022  \
region              city                                                        
Baden-Württemberg   Karlsruhe     0     0     0     0     0     0   490   467   
Bayern              München       0     0     0  1145  1170  1364  1983  1169   
Berlin              Berlin        0     0     0  4290  5405  5890  6207  6442   
Brandenburg         Potsdam       0     0     0     0   214   238   287   326   
Bremen              Bremen        0     0     0     0   654   631   623   721   
Hessen              Wiesbaden     0     0     0     0   159   235   304   256   
Nordrhein-Westfalen Köln          0     0     0  1037     0     0     0  1551   
                    Wuppertal     0     0     0     0     0     0     0   209   
Rheinland-Pfalz     Mainz         0     0     0   255   248   305   312   411   
Sachsen             Dresden       0     0     0     0     0   164   292   405   
Sachsen-Anhalt      Magdeburg   215   167   135   175   222   312   411   435   
Schleswig-Holstein  Kiel          0     0     0     0     0     0   272   341   
Thüringen           Erfurt        0     0     0     0   305   285   331   351   

year                           registrations  observations  incl?  capital  
region              city                                                    
Baden-Württemberg   Karlsruhe           True         False  False    False  
Bayern              München             True         False   True     True  
Berlin              Berlin              True          True   True     True  
Brandenburg         Potsdam             True         False   True     True  
Bremen              Bremen             False         False  False     True  
Hessen              Wiesbaden           True         False   True     True  
Nordrhein-Westfalen Köln                True         False  False    False  
                    Wuppertal           True         False  False    False  
Rheinland-Pfalz     Mainz               True         False   True     True  
Sachsen             Dresden             True         False   True     True  
Sachsen-Anhalt      Magdeburg           True          True   True     True  
Schleswig-Holstein  Kiel                True         False  False     True  
Thüringen           Erfurt             False         False  False     True

In [14]:
import os

# use geocoder to get the number of inhabitants for each city
import geocoder
from dotenv import load_dotenv

load_dotenv()


def get_population(city, region):
    g = geocoder.geonames(f"{city}, {region}", key=os.environ["GEONAMES_USERNAME"])
    return g.population


agg_df["kpop"] = agg_df.apply(lambda x: get_population(x.name[1], x.name[0]), axis=1)
agg_df

year                           2015  2016  2017  2018  2019  2020  2021  2022  \
region              city                                                        
Baden-Württemberg   Karlsruhe     0     0     0     0     0     0   490   467   
Bayern              München       0     0     0  1145  1170  1364  1983  1169   
Berlin              Berlin        0     0     0  4290  5405  5890  6207  6442   
Brandenburg         Potsdam       0     0     0     0   214   238   287   326   
Bremen              Bremen        0     0     0     0   654   631   623   721   
Hessen              Wiesbaden     0     0     0     0   159   235   304   256   
Nordrhein-Westfalen Köln          0     0     0  1037     0     0     0  1551   
                    Wuppertal     0     0     0     0     0     0     0   209   
Rheinland-Pfalz     Mainz         0     0     0   255   248   305   312   411   
Sachsen             Dresden       0     0     0     0     0   164   292   405   
Sachsen-Anhalt      Magdeburg   215   167   135   175   222   312   411   435   
Schleswig-Holstein  Kiel          0     0     0     0     0     0   272   341   
Thüringen           Erfurt        0     0     0     0   305   285   331   351   

year                           registrations  observations  incl?  capital  \
region              city                                                     
Baden-Württemberg   Karlsruhe           True         False  False    False   
Bayern              München             True         False   True     True   
Berlin              Berlin              True          True   True     True   
Brandenburg         Potsdam             True         False   True     True   
Bremen              Bremen             False         False  False     True   
Hessen              Wiesbaden           True         False   True     True   
Nordrhein-Westfalen Köln                True         False  False    False   
                    Wuppertal           True         False  False    False   
Rheinland-Pfalz     Mainz               True         False   True     True   
Sachsen             Dresden             True         False   True     True   
Sachsen-Anhalt      Magdeburg           True          True   True     True   
Schleswig-Holstein  Kiel                True         False  False     True   
Thüringen           Erfurt             False         False  False     True   

year                              kpop  
region              city                
Baden-Württemberg   Karlsruhe   283799  
Bayern              München    1260391  
Berlin              Berlin     3426354  
Brandenburg         Potsdam     182112  
Bremen              Bremen      546501  
Hessen              Wiesbaden   278609  
Nordrhein-Westfalen Köln        963395  
                    Wuppertal   360797  
Rheinland-Pfalz     Mainz       217123  
Sachsen             Dresden     556227  
Sachsen-Anhalt      Magdeburg   235775  
Schleswig-Holstein  Kiel        246601  
Thüringen           Erfurt      213692

In [15]:
agg_df.to_csv(path / "gpreg-overview.csv", index=True)

In [16]:
# calculate sums row (add later)
sums = agg_df.sum(numeric_only=False)
sums.name = ("", "sum", "")

In [17]:
# insert empty rows for regions that are not in the data
for region in german_regions:
    if region["name"] not in agg_df.index:
        agg_df.loc[(region["name"], "–"), :] = ""
        agg_df.loc[(region["name"], "–"), "kpop"] = "–"
# sort by region name
agg_df = agg_df.sort_index()
agg_df

,year,2015,2016,2017,2018,2019,2020,2021,2022,registrations,observations,incl?,capital,kpop
region,city,,,,,,,,,,,,,
Baden-Württemberg,Karlsruhe,0.0,0.0,0.0,0.0,0.0,0.0,490.0,467.0,True,False,False,False,283799.0
Bayern,München,0.0,0.0,0.0,1145.0,1170.0,1364.0,1983.0,1169.0,True,False,True,True,1260391.0
Berlin,Berlin,0.0,0.0,0.0,4290.0,5405.0,5890.0,6207.0,6442.0,True,True,True,True,3426354.0
Brandenburg,Potsdam,0.0,0.0,0.0,0.0,214.0,238.0,287.0,326.0,True,False,True,True,182112.0
Bremen,Bremen,0.0,0.0,0.0,0.0,654.0,631.0,623.0,721.0,False,False,False,True,546501.0
Hamburg,–,,,,,,,,,,,,,–
Hessen,Wiesbaden,0.0,0.0,0.0,0.0,159.0,235.0,304.0,256.0,True,False,True,True,278609.0
Mecklenburg-Vorpommern,–,,,,,,,,,,,,,–
Niedersachsen,–,,,,,,,,,,,,,–


In [18]:
# add sum row at the bottom
agg_df = pd.concat([agg_df, sums.to_frame().T])
agg_df

,year,2015,2016,2017,2018,2019,2020,2021,2022,registrations,observations,incl?,capital,kpop
region,city,,,,,,,,,,,,,
Baden-Württemberg,Karlsruhe,0.0,0.0,0.0,0.0,0.0,0.0,490.0,467.0,True,False,False,False,283799.0
Bayern,München,0.0,0.0,0.0,1145.0,1170.0,1364.0,1983.0,1169.0,True,False,True,True,1260391.0
Berlin,Berlin,0.0,0.0,0.0,4290.0,5405.0,5890.0,6207.0,6442.0,True,True,True,True,3426354.0
Brandenburg,Potsdam,0.0,0.0,0.0,0.0,214.0,238.0,287.0,326.0,True,False,True,True,182112.0
Bremen,Bremen,0.0,0.0,0.0,0.0,654.0,631.0,623.0,721.0,False,False,False,True,546501.0
Hamburg,–,,,,,,,,,,,,,–
Hessen,Wiesbaden,0.0,0.0,0.0,0.0,159.0,235.0,304.0,256.0,True,False,True,True,278609.0
Mecklenburg-Vorpommern,–,,,,,,,,,,,,,–
Niedersachsen,–,,,,,,,,,,,,,–


In [19]:
agg_df["#reg?"] = agg_df["registrations"].replace({0: "", 1: "✓"})
agg_df["#obs?"] = agg_df["observations"].replace({0: "", 1: "✓"})
agg_df["cap?"] = agg_df["capital"].replace({0: "", 1: "✓"})
agg_df["incl?"] = agg_df["incl?"].replace({0: "", 1: "✓"})
agg_df

,year,2015,2016,2017,2018,2019,2020,2021,2022,registrations,observations,incl?,capital,kpop,#reg?,#obs?,cap?
region,city,,,,,,,,,,,,,,,,
Baden-Württemberg,Karlsruhe,0.0,0.0,0.0,0.0,0.0,0.0,490.0,467.0,True,False,,False,283799.0,✓,,
Bayern,München,0.0,0.0,0.0,1145.0,1170.0,1364.0,1983.0,1169.0,True,False,✓,True,1260391.0,✓,,✓
Berlin,Berlin,0.0,0.0,0.0,4290.0,5405.0,5890.0,6207.0,6442.0,True,True,✓,True,3426354.0,✓,✓,✓
Brandenburg,Potsdam,0.0,0.0,0.0,0.0,214.0,238.0,287.0,326.0,True,False,✓,True,182112.0,✓,,✓
Bremen,Bremen,0.0,0.0,0.0,0.0,654.0,631.0,623.0,721.0,False,False,,True,546501.0,,,✓
Hamburg,–,,,,,,,,,,,,,–,,,
Hessen,Wiesbaden,0.0,0.0,0.0,0.0,159.0,235.0,304.0,256.0,True,False,✓,True,278609.0,✓,,✓
Mecklenburg-Vorpommern,–,,,,,,,,,,,,,–,,,
Niedersachsen,–,,,,,,,,,,,,,–,,,


In [20]:
# | label: data-official-overview
# | tbl-cap: Official data.
# | column: page

agg_df["kpop"] = agg_df["kpop"].apply(
    lambda x: f"{int(x/1000):,}" if x not in ["–", ""] else x
)
for col in ["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]:
    agg_df[col] = agg_df[col].apply(lambda x: f"{int(x):,}" if x != "" else "")
df["year"] = df["date"].dt.year.astype(str).apply(lambda x: x[:-2])
agg_df = agg_df.replace(0, "")
agg_df = agg_df.replace("0", "")
# reorder columns
agg_df = agg_df[
    [
        "kpop",
        "cap?",
        "#reg?",
        "#obs?",
        "2015",
        "2016",
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
        "2022",
        "incl?",
    ]
]
agg_df

,year,kpop,cap?,#reg?,#obs?,2015,2016,2017,2018,2019,2020,2021,2022,incl?
region,city,,,,,,,,,,,,,
Baden-Württemberg,Karlsruhe,283,,✓,,,,,,,,490,467,
Bayern,München,"1,260",✓,✓,,,,,"1,145","1,170","1,364","1,983","1,169",✓
Berlin,Berlin,"3,426",✓,✓,✓,,,,"4,290","5,405","5,890","6,207","6,442",✓
Brandenburg,Potsdam,182,✓,✓,,,,,,214,238,287,326,✓
Bremen,Bremen,546,✓,,,,,,,654,631,623,721,
Hamburg,–,–,,,,,,,,,,,,
Hessen,Wiesbaden,278,✓,✓,,,,,,159,235,304,256,✓
Mecklenburg-Vorpommern,–,–,,,,,,,,,,,,
Niedersachsen,–,–,,,,,,,,,,,,


In [21]:
agg_df_ = agg_df.copy()
agg_df_ = agg_df_.reset_index()
agg_df_ = agg_df_.replace("✓", "x").replace("", " ").astype(str)

agg_df_.to_csv(path / "gpreg-overview-pretty.csv", index=False, sep=";")
agg_df_

year,region,city,kpop,cap?,#reg?,#obs?,2015,2016,2017,2018,2019,2020,2021,2022,incl?
0,Baden-Württemberg,Karlsruhe,283,,x,,,,,,,,490,467,
1,Bayern,München,"1,260",x,x,,,,,"1,145","1,170","1,364","1,983","1,169",x
2,Berlin,Berlin,"3,426",x,x,x,,,,"4,290","5,405","5,890","6,207","6,442",x
3,Brandenburg,Potsdam,182,x,x,,,,,,214,238,287,326,x
4,Bremen,Bremen,546,x,,,,,,,654,631,623,721,
5,Hamburg,–,–,,,,,,,,,,,,
6,Hessen,Wiesbaden,278,x,x,,,,,,159,235,304,256,x
7,Mecklenburg-Vorpommern,–,–,,,,,,,,,,,,
8,Niedersachsen,–,–,,,,,,,,,,,,
9,Nordrhein-Westfalen,Köln,963,,x,,,,,"1,037",,,,"1,551",


In [22]:
from IPython.display import Markdown

table = pd.read_csv(path / "gpreg-overview-pretty.csv", sep=";").to_markdown(
    index=False
)
print(Markdown(table).data)

| region                 | city      | kpop   | cap?   | #reg?   | #obs?   | 2015   | 2016   | 2017   | 2018   | 2019   | 2020   | 2021   | 2022   | incl?   |
|:-----------------------|:----------|:-------|:-------|:--------|:--------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:-------|:--------|
| Baden-Württemberg      | Karlsruhe | 283    |        | x       |         |        |        |        |        |        |        | 490    | 467    |         |
| Bayern                 | München   | 1,260  | x      | x       |         |        |        |        | 1,145  | 1,170  | 1,364  | 1,983  | 1,169  | x       |
| Berlin                 | Berlin    | 3,426  | x      | x       | x       |        |        |        | 4,290  | 5,405  | 5,890  | 6,207  | 6,442  | x       |
| Brandenburg            | Potsdam   | 182    | x      | x       |         |        |        |        |        | 214    | 238    | 287    | 326    | x       |
| Bremen                 | Bremen    | 546    